# video-translation-service（Colab + T4 GPU）一键安装运行

目标：从直链下载 `1.mp4` → 生成中文字幕 `1_zh.srt`（不启用润色）。

注意：首次运行会下载 Whisper + 翻译模型（可能几 GB），需要等待一段时间。

In [1]:
#@title 0) 确认已启用 GPU（Runtime -> Change runtime type -> GPU -> T4）
!nvidia-smi -L

import sys
import torch

print("python:", sys.version)
print("torch:", torch.__version__)
print("cuda_available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("gpu:", torch.cuda.get_device_name(0))
else:
    raise RuntimeError("未检测到GPU：请在 Colab 切换到 GPU(T4) 运行时")


GPU 0: Tesla T4 (UUID: GPU-3dae4ecb-7813-1bb9-5604-d21cb086b3fd)
python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
torch: 2.9.0+cu126
cuda_available: True
gpu: Tesla T4


In [2]:
#@title 1) git clone 项目
# TODO: 改成你的仓库地址（私有仓库可用：https://<TOKEN>@github.com/<org>/<repo>.git）
REPO_URL = "https://github.com/MAE5blog/video-translation-service.git"
BRANCH = "main"
REPO_DIR = "/content/video-translation-service"

import os
import shutil

if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)

!git clone --depth 1 -b {BRANCH} {REPO_URL} {REPO_DIR}
%cd {REPO_DIR}


Cloning into '/content/video-translation-service'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 14 (delta 0), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), 39.87 KiB | 4.98 MiB/s, done.
/content/video-translation-service


In [3]:
#@title 2) 安装系统依赖（ffmpeg + 字体）
!apt-get update -y
!apt-get install -y ffmpeg fonts-noto-cjk libsndfile1
!ffmpeg -version | head -n 2


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable InRelease [3,917 B]               
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease   
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease    
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease                        
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]       
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,287 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204

In [4]:
#@title 3) 安装 Python 依赖
# 说明：Colab 自带 CUDA 版 torch，避免从 requirements.txt 里重复安装 torch（否则可能被换成CPU版/或耗时升级）
!python -m pip install -U pip
!grep -vE '^torch' requirements.txt > /tmp/requirements_no_torch.txt
!python -m pip install -r /tmp/requirements_no_torch.txt

# 可选：人声分离（Demucs）用于嘈杂/背景音乐场景
!python -m pip install demucs

# torchaudio 新版本保存音频需要 torchcodec（否则 Demucs 会报错）
!python -m pip install torchcodec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.2 MB/s eta 0:00:0000:01
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 74.9 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 156.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 33.6 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [faster-whisper]m [faster-whisper]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.5 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing bui

In [5]:
#@title 4) 生成 config.ini（GPU + 中文 + 不润色）
from pathlib import Path

config_text = """[API]
deepseek_api_key =

[Service]
host = 127.0.0.1
port = 50515

[Models]
# 如果显存不够/加载太慢，可改：asr_model_size = small
asr_model_size = medium

# T4 显存有限，默认使用 600M（更稳）；追求质量可改为 1.3B
translation_model = facebook/nllb-200-distilled-600M

use_gpu = true
beam_size = 3

[GPU]
# 在 GPU 重任务前清理 CUDA 缓存，降低 OOM 概率（略慢）
clear_cuda_cache_before_tasks = true

[Translation]
default_target_language = zh
use_deepseek_polish = false

[Audio]
# 人声分离（Demucs）：改善背景音乐/嘈杂场景识别
enable_vocal_separation = true
vocal_separation_model = htdemucs
# 默认用 cuda 加速；如遇 OOM 可改为 cpu
vocal_separation_device = cuda
"""

Path("config.ini").write_text(config_text, encoding="utf-8")
print("wrote config.ini")


wrote config.ini


In [6]:
#@title 5) 后台启动服务（不占用单元格）
import pathlib
import subprocess
import sys

import requests

SERVICE_URL = "http://127.0.0.1:50515"

def health():
    try:
        return requests.get(f"{SERVICE_URL}/health", timeout=2).json()
    except Exception:
        return None

h = health()
if not h:
    p = subprocess.Popen(
        [sys.executable, "server_optimized.py"],
        stdout=open("server.log", "wb"),
        stderr=subprocess.STDOUT,
        start_new_session=True,
    )
    pathlib.Path("server.pid").write_text(str(p.pid))
    print("server started, pid:", p.pid)
else:
    print("server already running:", h)


server started, pid: 1881


In [7]:
#@title 6) 等待模型加载完成（首次运行需要等待）
import json
import os
import time
from pathlib import Path

import requests

SERVICE_URL = "http://127.0.0.1:50515"
pid_path = Path("server.pid")
log_path = Path("server.log")

def pid_alive(pid: int) -> bool:
    try:
        os.kill(pid, 0)
        return True
    except Exception:
        return False

def tail_log(max_lines: int = 120) -> str:
    if not log_path.exists():
        return "(server.log not found)"
    try:
        lines = log_path.read_text(errors="ignore").splitlines()
        return "\n".join(lines[-max_lines:])
    except Exception as e:
        return f"(failed to read server.log: {e})"

for i in range(1800):  # 最多等 3600 秒
    try:
        h = requests.get(f"{SERVICE_URL}/health", timeout=5).json()
    except Exception as e:
        if i % 5 == 0:
            print(f"{i*2:>4}s", "waiting for server...", repr(e))
        if pid_path.exists():
            try:
                pid = int(pid_path.read_text().strip() or "0")
            except Exception:
                pid = 0
            if pid and not pid_alive(pid):
                print("\nserver process exited; last logs:\n")
                print(tail_log())
                raise
        time.sleep(2)
        continue

    if h.get("ready"):
        print("READY:\n", json.dumps(h, ensure_ascii=False, indent=2))
        break
    if i % 5 == 0:
        print(f"{i*2:>4}s", h.get("phase"), h.get("progress"), h.get("message"))
    if h.get("phase") == "error":
        raise RuntimeError(h.get("error") or "模型加载失败，请查看 server.log")
    time.sleep(2)
else:
    raise TimeoutError("等待服务就绪超时：请查看 server.log")


   0s waiting for server... ConnectionError(MaxRetryError("HTTPConnectionPool(host='127.0.0.1', port=50515): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x79608e587e30>: Failed to establish a new connection: [Errno 111] Connection refused'))"))
  10s waiting for server... ConnectionError(MaxRetryError("HTTPConnectionPool(host='127.0.0.1', port=50515): Max retries exceeded with url: /health (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x795f9c096480>: Failed to establish a new connection: [Errno 111] Connection refused'))"))
  20s asr_loading 0.08 Loading ASR model medium ...
  30s asr_loading 0.08 Loading ASR model medium ...
  40s translation_loading 0.55 Loading translation model facebook/nllb-200-distilled-600M ...
  50s translation_loading 0.55 Loading translation model facebook/nllb-200-distilled-600M ...
  60s translation_loading 0.55 Loading translation model facebook/nllb-200-d

In [8]:
#@title 7) 下载测试视频（自动保存为 1.mp4）
# 直接把你的直链贴到这里即可
VIDEO_URL = "https://oplist.mae5.com/d/本地存储/1.mp4?sign=b58L3c5JYGAMNwLcO09asS9CV8aHTlGBXiO3Yi8Pe0Y=:0"
OUT_PATH = "1.mp4"
FORCE = False  # True=总是重新下载

import urllib.parse
from pathlib import Path

import requests

try:
    from tqdm.auto import tqdm
except Exception:
    tqdm = None

def normalize_url(url: str) -> str:
    parts = urllib.parse.urlsplit(url)
    # 对 path 做编码，避免包含中文路径时部分工具报错
    path = urllib.parse.quote(parts.path)
    return urllib.parse.urlunsplit((parts.scheme, parts.netloc, path, parts.query, parts.fragment))

out = Path(OUT_PATH)
if out.exists() and out.stat().st_size > 0 and not FORCE:
    print("exists:", out, out.stat().st_size)
else:
    url = normalize_url(VIDEO_URL)
    with requests.get(url, stream=True, timeout=60) as r:
        r.raise_for_status()
        total = int(r.headers.get("content-length") or 0)
        if tqdm and total > 0:
            pbar = tqdm(total=total, unit="B", unit_scale=True, desc="download")
        else:
            pbar = None
        with open(out, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024 * 1024):
                if not chunk:
                    continue
                f.write(chunk)
                if pbar:
                    pbar.update(len(chunk))
        if pbar:
            pbar.close()
    print("downloaded:", out, out.stat().st_size)


download:   0%|          | 0.00/62.9M [00:00<?, ?B/s]

downloaded: 1.mp4 62864501


In [9]:
#@title 8) 翻译生成中文字幕（不润色）
!python batch_translate.py 1.mp4 -t zh --translation-only


日志文件: log/translation_20251230_062027.log
开始时间: 2025-12-30 06:20:27

✓ 翻译服务正常运行


处理: 1.mp4
  [1/5] 提取音频...
  [1/5] 提取音频完成 ✓
    [GPU] 已清理缓存: before_vocal_separation（free 10.9GB / total 14.7GB）
  [2/5] 人声分离（Demucs，可能较慢）...
    [Demucs]  22% |#####...................| 2s    [Demucs] 进度: 22%
    [Demucs] 100% |########################| 3s    [Demucs] 进度: 100%
    [Demucs] 100% |########################| 3s    [Demucs] 进度: 100%
    [Demucs] 100% |########################| 4s    [Demucs] 进度: 100%
    [Demucs] 100% |########################| 4s    [Demucs] 进度: 100%
    [Demucs] 100% |########################| 5s    [Demucs] 进度: 100%
    [Demucs] 100% |########################| 5s    [Demucs] 进度: 100%
    [Demucs] 100% |########################| 6s    [Demucs] 进度: 100%
    [Demucs] 100% |########################| 6s    [Demucs] 进度: 100%
    [Demucs] 100% |########################| 7s    [Demucs] 进度: 100%
    [Demucs]  10% |##......................| 16s    [Demucs] 进度: 10%
    [Demucs]  20% |

In [ ]:
#@title 9) 合并字幕与视频（生成带字幕 MP4）
import subprocess
from pathlib import Path

VIDEO_PATH = Path("1.mp4")
SRT_PATH = Path("1_zh.srt")
MODE = "hard"  # hard=硬字幕(烧录)；soft=软字幕(可开关)

if not VIDEO_PATH.exists():
    raise FileNotFoundError(VIDEO_PATH)
if not SRT_PATH.exists():
    raise FileNotFoundError(SRT_PATH)

OUT_PATH = Path("1_zh_subbed.mp4")

if MODE == "soft":
    cmd = [
        "ffmpeg",
        "-y",
        "-i",
        str(VIDEO_PATH),
        "-i",
        str(SRT_PATH),
        "-map",
        "0:v",
        "-map",
        "0:a?",
        "-map",
        "1:0",
        "-c:v",
        "copy",
        "-c:a",
        "copy",
        "-c:s",
        "mov_text",
        str(OUT_PATH),
    ]
else:
    encoders = subprocess.check_output(
        ["ffmpeg", "-hide_banner", "-encoders"],
        text=True,
        stderr=subprocess.STDOUT,
    )
    use_nvenc = "h264_nvenc" in encoders

    vf = "subtitles=1_zh.srt:charenc=UTF-8:force_style='FontName=Noto Sans CJK SC,FontSize=24,Outline=1,Shadow=1'"
    cmd = [
        "ffmpeg",
        "-y",
        "-i",
        str(VIDEO_PATH),
        "-vf",
        vf,
        "-c:a",
        "copy",
    ]
    if use_nvenc:
        cmd += ["-c:v", "h264_nvenc", "-preset", "p4", "-cq", "19"]
    else:
        cmd += ["-c:v", "libx264", "-preset", "fast", "-crf", "19"]
    cmd.append(str(OUT_PATH))

print("Running:\n ", " ".join(cmd))
subprocess.run(cmd, check=True)
print("OK:", OUT_PATH)


In [ ]:
#@title 10) 下载生成的字幕 / 带字幕视频
from google.colab import files

files.download("1_zh.srt")
files.download("1_zh_subbed.mp4")


In [ ]:
#@title 11) （可选）停止服务
import os
import signal
import pathlib

pid_path = pathlib.Path("server.pid")
if pid_path.exists():
    pid = int(pid_path.read_text().strip())
    os.kill(pid, signal.SIGTERM)
    print("killed:", pid)
else:
    print("server.pid not found")
